### Selected Word Group In Youtube Sentence

In [50]:
import os
import re
import pandas as pd
import numpy as np
import glob
from functools import reduce
from pathlib import Path
import shutil
from itertools import islice

In [51]:
# language pair
lang_folder = "Turkish"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> target language for learner
#lang_pair = "Arabic"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> native language

# native word select for Part 2 
word_start = 0  # 0  # native word start index
word_end = 200 # 28  # native word end index

# source path
source_path = "/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Turkish/Deployment2/Result/Deploy2 Result Manuel/Turkish"

# parameter
file = "Basics Cover the Most Master File Threegram Selected 28 Word"  # without xlsx extention
sheets = "Sheet1"  # 2 gram target, 2 gram hybrid
file_ext = "Threegram_28_Word"
first_column = 0  # start column location number
end_column = 1
time_shift = 0.6
sample_num = 6

# youtube sentence condition
sent_time_start = True  # True, False
sent_time_start_time = 700 # sn

In [52]:
path = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/\
Deployment2/Result/3-Selected Word Group In Youtube Sentence/{lang_folder.capitalize()}"

Path(path).mkdir(parents=True, exist_ok=True)

In [53]:
def word_group_youtube(df, search_list, target_column, sample_num):
    '''
    word_group_youtube(df_youtube_sentence, search_list, "sentence", 6)
    ''' 
    df_search_result = pd.DataFrame()
    for j in search_list:
        try:
            df_select = df[df[f"{target_column}"].str.contains(fr"(?:\s|^){j}(?:\s|$)", na=True)].sample(sample_num)
        except:
            df_select = df[df[f"{target_column}"].str.contains(fr"(?:\s|^){j}(?:\s|$)", na=True)].head(sample_num)
        #df_result = df[df[f"{target_column}"].str.contains(fr"(?:\s|^){j}(?:\s|$)", na=True)]  # sentence length part
        #df_result.sort_values(f"{target_column}",key=lambda x:x.str.len(), inplace=True)
        #df_select = df_result.head(sample_num)
        df_select.insert(0,"search_string",j)
        df_search_result = pd.concat([df_search_result,df_select], axis=0)
    df_search_result.reset_index(inplace=True, drop=True)
    
    return df_search_result

In [54]:
def word_group_time_loc(df, search, start_sent, end_sent, sent, sent_video_id):
    '''
    word_group_time_loc(df_search_result, "search_string", "start_time", "end_time", "sentence", "video_id")
    '''
    df.reset_index(drop=True, inplace=True)
    word_time_loc_list = []
    for i in range(len(df)):
        word = df.loc[i,f"{search}"]
        start_time = df.loc[i,f"{start_sent}"]
        end_time = df.loc[i,f"{end_sent}"]
        sentence = df.loc[i,f"{sent}"]
        video_id = df.loc[i,f"{sent_video_id}"]
        time_length = end_time-start_time
        sentence_length = len(sentence)
        time_length_ratio = time_length/sentence_length
        loc_list = []
        for j in re.finditer(fr"(?:\s|^){word}(?:\s|$)", sentence, re.IGNORECASE|re.UNICODE):
            loc_list.append(j)
            start = loc_list[0].start()
            end = loc_list[0].end()
            start_loc = start_time+(start*time_length_ratio)
            end_loc = start_time+(end*time_length_ratio)
        word_time_loc_list.append([word,start_loc,end_loc,sentence,video_id])
    df_word_time_loc = pd.DataFrame(word_time_loc_list, columns=[f"{search}",f"{start_sent}",f"{end_sent}",f"{sent}",f"{sent_video_id}"])

    return df_word_time_loc

In [55]:
def word_count_result(df, column_list, set_condition=False): # df is dataframe, column_list is list value
    '''
    word_count_bool(df, column_list): df columns word count for word frequency\n
    df is dataframe, column_list is list value\n
    word_count_bool(df, ["word","twogram"]):
    '''
    list_all = []
    for i in df.loc[:,[x for x in column_list]].columns:
        if set_condition:
            var_list = set(df[f"{i}"].dropna().tolist())
        else:
            var_list = df[f"{i}"].dropna().tolist()
        for j in var_list:
            list_all.append(j)
    text = " ".join(list_all)
    word_list = re.findall(r"\w+",text, re.UNICODE)
    df_word_list = pd.DataFrame(word_list, columns=["word"])
    #df_word_list.rename(columns={0:"word"}, inplace=True)
    df_word_count = pd.DataFrame(df_word_list.value_counts())
    df_word_count.reset_index(inplace=True)
    df_word_count.rename(columns={0:"word_count"}, inplace=True)
    df_word_count.sort_values("word_count", ascending=False, inplace=True)
    
    return  df_word_count

In [56]:
df_word_all = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Deployment/Data/Word/Word_Merge_Preprocess.xlsx")
df_word_all

,word,frequency
0,bir,18835735
1,bu,11062659
2,ne,8025880
3,ve,7766036
4,için,5484109
...,...,...
987924,karneleme,5
987925,karnaya,5
987926,dörtlümüzün,5
987927,karnavalınız,5


In [57]:
df_word_select = df_word_all.iloc[word_start:word_end,]
df_word_select

,word,frequency
0,bir,18835735
1,bu,11062659
2,ne,8025880
3,ve,7766036
4,için,5484109
...,...,...
195,saat,399989
196,onunla,399330
197,yapıyorsun,398274
198,neler,397377


In [58]:
df_youtube_sentence = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Youtube/Result/{lang_folder.capitalize()}/Sentence Clean Merge/Clean_Youtube_Sentence_Merge_Result.csv")
df_youtube_sentence

,start_time,end_time,sentence,video_id
0,00:02:51.948,00:02:58.829,özgür bunlar normalde kamyon daha büyük araçla...,8V9tq1pe8eI
1,00:03:00.956,00:03:04.236,burcu arka tarafı bağlamak kolay olmayacak,8V9tq1pe8eI
2,00:03:13.434,00:03:16.327,özgür arabaya yarım tur attıracağım,8V9tq1pe8eI
3,00:03:17.235,00:03:21.338,burcu biraz daha devam et devam et tamam oldu,8V9tq1pe8eI
4,00:03:27.806,00:03:33.383,burcu şimdilik iki tekere takacağız ama kar ka...,8V9tq1pe8eI
...,...,...,...,...
3063204,00:11:07.990,00:11:14.560,oynatma listesi linkini videonun sağ üst köşes...,MvWp9pWLihA
3063205,00:11:14.560,00:11:21.040,kısmına ekledim dilediğiniz kenar deseni uyarl...,MvWp9pWLihA
3063206,00:11:21.040,00:11:27.880,ibaretti bir sonraki farklı bir elle örgü eğit...,MvWp9pWLihA
3063207,00:11:27.880,00:11:32.290,kanalıma abone olarak videomu beğenmeyi unutma...,MvWp9pWLihA


In [59]:
df_youtube_sentence['start_time'] = pd.to_timedelta(df_youtube_sentence['start_time']) # data type converted timedelta for second 
df_youtube_sentence['end_time'] = pd.to_timedelta(df_youtube_sentence['end_time'])

In [60]:
df_youtube_sentence['start_time'] = df_youtube_sentence['start_time'].apply(lambda x: x.total_seconds()) # convert seconds
df_youtube_sentence['end_time'] = df_youtube_sentence['end_time'].apply(lambda x: x.total_seconds())
df_youtube_sentence

,start_time,end_time,sentence,video_id
0,171.948,178.829,özgür bunlar normalde kamyon daha büyük araçla...,8V9tq1pe8eI
1,180.956,184.236,burcu arka tarafı bağlamak kolay olmayacak,8V9tq1pe8eI
2,193.434,196.327,özgür arabaya yarım tur attıracağım,8V9tq1pe8eI
3,197.235,201.338,burcu biraz daha devam et devam et tamam oldu,8V9tq1pe8eI
4,207.806,213.383,burcu şimdilik iki tekere takacağız ama kar ka...,8V9tq1pe8eI
...,...,...,...,...
3063204,667.990,674.560,oynatma listesi linkini videonun sağ üst köşes...,MvWp9pWLihA
3063205,674.560,681.040,kısmına ekledim dilediğiniz kenar deseni uyarl...,MvWp9pWLihA
3063206,681.040,687.880,ibaretti bir sonraki farklı bir elle örgü eğit...,MvWp9pWLihA
3063207,687.880,692.290,kanalıma abone olarak videomu beğenmeyi unutma...,MvWp9pWLihA


In [61]:
if sent_time_start:
    df_youtube_select = df_youtube_sentence[df_youtube_sentence["start_time"] <= sent_time_start_time]
    df_youtube_select.reset_index(drop=True, inplace=True)
else:
    df_youtube_select = df_youtube_sentence

df_youtube_select

,start_time,end_time,sentence,video_id
0,171.948,178.829,özgür bunlar normalde kamyon daha büyük araçla...,8V9tq1pe8eI
1,180.956,184.236,burcu arka tarafı bağlamak kolay olmayacak,8V9tq1pe8eI
2,193.434,196.327,özgür arabaya yarım tur attıracağım,8V9tq1pe8eI
3,197.235,201.338,burcu biraz daha devam et devam et tamam oldu,8V9tq1pe8eI
4,207.806,213.383,burcu şimdilik iki tekere takacağız ama kar ka...,8V9tq1pe8eI
...,...,...,...,...
1050454,667.990,674.560,oynatma listesi linkini videonun sağ üst köşes...,MvWp9pWLihA
1050455,674.560,681.040,kısmına ekledim dilediğiniz kenar deseni uyarl...,MvWp9pWLihA
1050456,681.040,687.880,ibaretti bir sonraki farklı bir elle örgü eğit...,MvWp9pWLihA
1050457,687.880,692.290,kanalıma abone olarak videomu beğenmeyi unutma...,MvWp9pWLihA


In [62]:
df_word_group_select = pd.read_excel(f"{source_path}/{file}.xlsx", sheet_name= f"{sheets}")
df_word_group_select

,threegram
0,ama ne için
1,bana ne olacak
2,bana ne oldu
3,başka bir şey
4,başka ne var
...,...
93,ve o da
94,ve sen de
95,yok değil mi
96,yoksa ben mi


In [ ]:
# Option
remove_videoid_list = list(set(df_word_group_select["video_id"]))
#remove_videoid_list

In [ ]:
len(remove_videoid_list)

In [ ]:
# Option
df_youtube_select = df_youtube_select[~df_youtube_select["video_id"].isin(remove_videoid_list)]
df_youtube_select

In [63]:
search_list = []
for i in range(len(df_word_group_select)):
    for j in df_word_group_select.columns[first_column:end_column]:
        string = df_word_group_select.loc[i,j]
        
        if pd.isnull(string) == False and string != 'nan':
            search_list.append(string.strip())
            search_list = list(set(search_list))  # for unique value
        else:
            pass

In [64]:
len(search_list)

98

In [65]:
df_search_result = word_group_youtube(df_youtube_select, search_list, "sentence", sample_num)
df_search_result

,search_string,start_time,end_time,sentence,video_id
0,sen de öyle,72.520,74.140,sen de öyle yapmıyor musun,jRjkpgSqjLM
1,sen de öyle,112.875,115.152,o pasta buraya gelmeyecek sen de öyle,6TGH_yi9JiQ
2,sen de öyle,291.687,293.706,ekremciğim sen de öyle geç istersen,D55MAAuexLk
3,sen de öyle,422.260,423.310,sen de öyle başlıyor musun,h07c3ds0dGw
4,sen de öyle,137.610,139.577,sen de öyle yap kurcalamayı bırak,f2RhCZ9oGY4
...,...,...,...,...,...
503,bu çok güzel,202.320,203.580,bu çok güzel bir fikir,JT9bt6AwEuc
504,bu çok güzel,39.520,45.780,hiç birinde ingilizce de açıklama yok bir şunu...,AXatSyborAM
505,bu çok güzel,316.220,322.120,heh bu bacak lazımdı bize bu çok güzel oldu yo...,dSG4sD2bmXg
506,bu çok güzel,175.250,181.260,göstermeden geçemedim ilk kez elektrik direkle...,8AQtR8gCBhc


In [66]:
df_word_group_time_loc_result = word_group_time_loc(df_search_result, "search_string", "start_time", "end_time", "sentence", "video_id")
df_word_group_time_loc_result

,search_string,start_time,end_time,sentence,video_id
0,sen de öyle,72.520000,73.267692,sen de öyle yapmıyor musun,jRjkpgSqjLM
1,sen de öyle,114.413514,115.152000,o pasta buraya gelmeyecek sen de öyle,6TGH_yi9JiQ
2,sen de öyle,292.263857,293.013771,ekremciğim sen de öyle geç istersen,D55MAAuexLk
3,sen de öyle,422.260000,422.744615,sen de öyle başlıyor musun,h07c3ds0dGw
4,sen de öyle,137.610000,138.325273,sen de öyle yap kurcalamayı bırak,f2RhCZ9oGY4
...,...,...,...,...,...
503,bu çok güzel,202.320000,203.064545,bu çok güzel bir fikir,JT9bt6AwEuc
504,bu çok güzel,42.940412,43.843918,hiç birinde ingilizce de açıklama yok bir şunu...,AXatSyborAM
505,bu çok güzel,318.421493,319.654328,heh bu bacak lazımdı bize bu çok güzel oldu yo...,dSG4sD2bmXg
506,bu çok güzel,180.217449,181.076020,göstermeden geçemedim ilk kez elektrik direkle...,8AQtR8gCBhc


In [67]:
df_word_group_time_loc_result.start_time = df_word_group_time_loc_result.start_time.apply(lambda x: (x-time_shift))
df_word_group_time_loc_result.end_time = df_word_group_time_loc_result.end_time.apply(lambda x: (x+time_shift))
df_word_group_time_loc_result

,search_string,start_time,end_time,sentence,video_id
0,sen de öyle,71.920000,73.867692,sen de öyle yapmıyor musun,jRjkpgSqjLM
1,sen de öyle,113.813514,115.752000,o pasta buraya gelmeyecek sen de öyle,6TGH_yi9JiQ
2,sen de öyle,291.663857,293.613771,ekremciğim sen de öyle geç istersen,D55MAAuexLk
3,sen de öyle,421.660000,423.344615,sen de öyle başlıyor musun,h07c3ds0dGw
4,sen de öyle,137.010000,138.925273,sen de öyle yap kurcalamayı bırak,f2RhCZ9oGY4
...,...,...,...,...,...
503,bu çok güzel,201.720000,203.664545,bu çok güzel bir fikir,JT9bt6AwEuc
504,bu çok güzel,42.340412,44.443918,hiç birinde ingilizce de açıklama yok bir şunu...,AXatSyborAM
505,bu çok güzel,317.821493,320.254328,heh bu bacak lazımdı bize bu çok güzel oldu yo...,dSG4sD2bmXg
506,bu çok güzel,179.617449,181.676020,göstermeden geçemedim ilk kez elektrik direkle...,8AQtR8gCBhc


In [68]:
df_word_group_time_loc_result.start_time = df_word_group_time_loc_result.start_time.apply(lambda x: round(x))
df_word_group_time_loc_result.end_time = df_word_group_time_loc_result.end_time.apply(lambda x: round(x))
df_word_group_time_loc_result 

,search_string,start_time,end_time,sentence,video_id
0,sen de öyle,72,74,sen de öyle yapmıyor musun,jRjkpgSqjLM
1,sen de öyle,114,116,o pasta buraya gelmeyecek sen de öyle,6TGH_yi9JiQ
2,sen de öyle,292,294,ekremciğim sen de öyle geç istersen,D55MAAuexLk
3,sen de öyle,422,423,sen de öyle başlıyor musun,h07c3ds0dGw
4,sen de öyle,137,139,sen de öyle yap kurcalamayı bırak,f2RhCZ9oGY4
...,...,...,...,...,...
503,bu çok güzel,202,204,bu çok güzel bir fikir,JT9bt6AwEuc
504,bu çok güzel,42,44,hiç birinde ingilizce de açıklama yok bir şunu...,AXatSyborAM
505,bu çok güzel,318,320,heh bu bacak lazımdı bize bu çok güzel oldu yo...,dSG4sD2bmXg
506,bu çok güzel,180,182,göstermeden geçemedim ilk kez elektrik direkle...,8AQtR8gCBhc


In [69]:
df_word_group_time_loc_result["video_url"] = "https://www.youtube.com/watch?v="+df_word_group_time_loc_result['video_id'].map(str)+"&t="+df_word_group_time_loc_result['start_time'].map(str)+"s"
df_word_group_time_loc_result

,search_string,start_time,end_time,sentence,video_id,video_url
0,sen de öyle,72,74,sen de öyle yapmıyor musun,jRjkpgSqjLM,https://www.youtube.com/watch?v=jRjkpgSqjLM&t=72s
1,sen de öyle,114,116,o pasta buraya gelmeyecek sen de öyle,6TGH_yi9JiQ,https://www.youtube.com/watch?v=6TGH_yi9JiQ&t=...
2,sen de öyle,292,294,ekremciğim sen de öyle geç istersen,D55MAAuexLk,https://www.youtube.com/watch?v=D55MAAuexLk&t=...
3,sen de öyle,422,423,sen de öyle başlıyor musun,h07c3ds0dGw,https://www.youtube.com/watch?v=h07c3ds0dGw&t=...
4,sen de öyle,137,139,sen de öyle yap kurcalamayı bırak,f2RhCZ9oGY4,https://www.youtube.com/watch?v=f2RhCZ9oGY4&t=...
...,...,...,...,...,...,...
503,bu çok güzel,202,204,bu çok güzel bir fikir,JT9bt6AwEuc,https://www.youtube.com/watch?v=JT9bt6AwEuc&t=...
504,bu çok güzel,42,44,hiç birinde ingilizce de açıklama yok bir şunu...,AXatSyborAM,https://www.youtube.com/watch?v=AXatSyborAM&t=42s
505,bu çok güzel,318,320,heh bu bacak lazımdı bize bu çok güzel oldu yo...,dSG4sD2bmXg,https://www.youtube.com/watch?v=dSG4sD2bmXg&t=...
506,bu çok güzel,180,182,göstermeden geçemedim ilk kez elektrik direkle...,8AQtR8gCBhc,https://www.youtube.com/watch?v=8AQtR8gCBhc&t=...


In [70]:
search_result_num = df_word_group_time_loc_result["search_string"].nunique()
search_result_num

93

In [89]:
search_result_set = set(df_word_group_time_loc_result["search_string"])
search_all_set = set(search_list)

In [90]:
search_all_set.difference(search_result_set)

{'bunun için üzgünüm',
 'hayır ben değil',
 'hayır ben sadece',
 'o ne için',
 'şimdi ne var'}

In [71]:
df_result_word_manuel_count = word_count_result(df_word_group_time_loc_result, df_word_group_time_loc_result.columns[0:1], set_condition=True)
df_result_word_manuel_count

,word,word_count
0,bu,26
1,ne,17
2,çok,15
3,için,15
4,de,14
5,mi,14
6,bir,13
7,kadar,12
8,var,11
9,da,11


In [80]:
# # Option merge
# df_word_count_merge = pd.merge(df_result_word_manuel_count, df_word_select[["word"]], how="inner", on="word")
# df_word_count_merge.drop_duplicates(inplace=True)
# df_word_count_merge.sort_values(by="word_count", ascending=False, inplace=True)
# df_word_count_merge.reset_index(drop=True, inplace=True)
# df_word_count_merge

In [81]:
set_word = set(df_word_select["word"])
#set_count = set(df_word_count_merge["word"])  # Option merge
set_count = set(df_result_word_manuel_count["word"])

In [82]:
set_word.difference(set_count)

{'adam',
 'al',
 'anne',
 'artık',
 'aslında',
 'aynı',
 'baba',
 'bak',
 'bakalım',
 'belki',
 'bence',
 'beni',
 'benimle',
 'bile',
 'biliyor',
 'biliyorum',
 'bilmiyorum',
 'biraz',
 'birkaç',
 'birlikte',
 'biz',
 'bize',
 'bizi',
 'bizim',
 'bugün',
 'buna',
 'burada',
 'buraya',
 'bütün',
 'büyük',
 'dakika',
 'devam',
 'değilim',
 'diye',
 'dostum',
 'doğru',
 'dur',
 'ederim',
 'efendim',
 'en',
 'et',
 'eğer',
 'gece',
 'gel',
 'geldi',
 'geliyor',
 'gerek',
 'geri',
 'gerçek',
 'gerçekten',
 'gibi',
 'git',
 'göre',
 'gün',
 'hadi',
 'hakkında',
 'hala',
 'harika',
 'haydi',
 'hemen',
 'her',
 'herkes',
 'hey',
 'hiçbir',
 'iki',
 'ile',
 'ilk',
 'ister',
 'istiyorsun',
 'istiyorum',
 'izin',
 'içinde',
 'iş',
 'işte',
 'kendi',
 'küçük',
 'kız',
 'lazım',
 'lütfen',
 'merhaba',
 'misin',
 'mu',
 'musun',
 'mü',
 'mısın',
 'nasıl',
 'neler',
 'nerede',
 'oh',
 'ol',
 'olabilir',
 'olarak',
 'olduğunu',
 'olmak',
 'olmaz',
 'olsun',
 'olur',
 'oluyor',
 'ona',
 'onlar',
 'onl

In [75]:
df_word_group_time_loc_result.to_excel(f"{lang_folder.capitalize()}_{search_result_num}_{file_ext}_\
{sample_num}_Youtube_{time_shift}s_Timeshift_Result.xlsx", index=False) 

In [83]:
# df_result_word_manuel_count.to_excel(f"{lang_folder.capitalize()}_{search_result_num}_{file_ext}_\
# {sample_num}_Youtube_{time_shift}s_Timeshift_Result_Word_Count.xlsx", index=False) 

#### Copy Move And Delete

In [84]:
output_file = glob.glob(f"{lang_folder.capitalize()}_*_{sample_num}_Youtube_{time_shift}s_Timeshift_Result*.xlsx")
output_file

['Turkish_93_Threegram_28_Word_6_Youtube_0.6s_Timeshift_Result_Word_Count.xlsx']

In [85]:
for y in output_file:
    source = y # source directory
    destination = path
    shutil.copy2(source, destination)

In [86]:
for z in output_file:
    try:
        os.remove(z)
    except:
        pass

#### Temp

In [ ]:
set_result = set(df_word_group_time_loc_result["search_string"].unique())
set_selected = set(search_list)

In [ ]:
df_not_in_result = pd.DataFrame(set_selected.difference(set_result), columns=["not_in_youtube_result"])
df_not_in_result

In [ ]:
not_in_result_num = len(df_not_in_result)
not_in_result_num

In [ ]:
#df_not_in_result.to_excel(f"{lang_folder.capitalize()}_{not_in_result_num}_{file_ext}_\
#Not_In_{sample_num}_Youtube_{time_shift}s_Timeshift_Result.xlsx", index=False) 

In [ ]:
search_list2 = df_not_in_result["not_in_youtube_result"].to_list()

In [ ]:
df_search_result2 = word_group_youtube(df_youtube_sentence, search_list2, "sentence", sample_num)
df_search_result2

In [ ]:
search_result2_num = df_search_result2["search_string"].nunique()
search_result2_num